In [ ]:
import os
import copy
import pandas as pd
import numpy as np
from collections import Counter
from datetime import date, timedelta
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

## Data Preparation

* Loading the original data

In [ ]:
%%time
# Around 1mins
xlsx = pd.ExcelFile('data-Test-2020-09-01.xlsx')
sheets = pd.read_excel(xlsx, sheet_name=None, index_col=0, 
                       na_filter=True, convert_float=False)

met_stations = sheets['met-stations']
met_real = sheets['met-real']
met_forecast = sheets['met-forecast'] 
rl_sites = sheets['rl-sites']
rl_kpis = sheets['rl-kpis']
distances = sheets['distances']

# Station names
stations = met_stations['station_no'].tolist()

# Forecast dates
forecast_dates = sorted(list(set(met_forecast['datetime'])))

CPU times: user 25.2 s, sys: 254 ms, total: 25.5 s
Wall time: 25.5 s


#### On MET Forecast
* One value per each (station_no, datetime)
* The value will be the mean of the values per each (station_no, datetime)
* One hot encoding for the categorical data
* **Only the forecast of the next day would be consider**

*Numerical data*

In [ ]:
met_forecast.sort_values(by=['station_no', 'datetime']).head()

,station_no,datetime,report_time,weather_day1,temp_max_day1,temp_min_day1,humidity_max_day1,humidity_min_day1,wind_dir_day1,wind_speed_day1,weather_day2,temp_max_day2,temp_min_day2,humidity_max_day2,humidity_min_day2,wind_dir_day2,wind_speed_day2,weather_day3,temp_max_day3,temp_min_day3,humidity_max_day3,humidity_min_day3,wind_dir_day3,wind_speed_day3,weather_day4,temp_max_day4,temp_min_day4,humidity_max_day4,humidity_min_day4,wind_dir_day4,wind_speed_day4,weather_day5,temp_max_day5,temp_min_day5,humidity_max_day5,humidity_min_day5,wind_dir_day5,wind_speed_day5
0.0,WS_17038,2020-01-02,evening,rain,10.0,7.0,NaN,NaN,NaN,NaN,overcast clouds,11.0,7.0,NaN,NaN,NaN,NaN,scattered clouds,11.0,8.0,NaN,NaN,NaN,NaN,overcast clouds,14.0,7.0,NaN,NaN,NaN,NaN,overcast clouds,16.0,12.0,NaN,NaN,NaN,NaN
1.0,WS_17038,2020-01-02,morning,rain,10.0,7.0,NaN,NaN,NaN,NaN,overcast clouds,11.0,7.0,NaN,NaN,NaN,NaN,scattered clouds,11.0,8.0,NaN,NaN,NaN,NaN,overcast clouds,14.0,7.0,NaN,NaN,NaN,NaN,overcast clouds,16.0,12.0,NaN,NaN,NaN,NaN
96.0,WS_17038,2020-01-03,evening,rain,10.0,7.0,78.0,68.0,322.0,11.0,scattered clouds,11.0,8.0,84.0,67.0,23.0,9.0,overcast clouds,12.0,7.0,80.0,61.0,169.0,12.0,overcast clouds,14.0,10.0,63.0,45.0,178.0,13.0,rain,11.0,8.0,83.0,63.0,288.0,32.0
97.0,WS_17038,2020-01-03,morning,overcast clouds,10.0,7.0,78.0,68.0,322.0,11.0,scattered clouds,11.0,8.0,84.0,67.0,23.0,9.0,overcast clouds,12.0,7.0,80.0,61.0,169.0,12.0,overcast clouds,14.0,10.0,63.0,45.0,178.0,13.0,NaN,11.0,8.0,83.0,63.0,288.0,32.0
192.0,WS_17038,2020-01-04,evening,scattered clouds,10.0,7.0,85.0,65.0,41.0,11.0,scattered clouds,12.0,6.0,83.0,62.0,126.0,11.0,scattered clouds,13.0,10.0,62.0,48.0,140.0,14.0,rain,12.0,9.0,81.0,50.0,283.0,25.0,rain,9.0,6.0,86.0,72.0,287.0,38.0


In [ ]:
to_drop = [c for c in met_forecast.columns 
           if ('day1' not in c) and (c not in ['station_no', 'datetime', 'report_time'])]
met_forecast_v2 = met_forecast.drop(columns=to_drop)
met_forecast_v2.head()

,station_no,datetime,report_time,weather_day1,temp_max_day1,temp_min_day1,humidity_max_day1,humidity_min_day1,wind_dir_day1,wind_speed_day1
0.0,WS_17038,2020-01-02,evening,rain,10.0,7.0,NaN,NaN,NaN,NaN
1.0,WS_17038,2020-01-02,morning,rain,10.0,7.0,NaN,NaN,NaN,NaN
2.0,WS_17232,2020-01-02,evening,scattered clouds,12.0,5.0,NaN,NaN,NaN,NaN
3.0,WS_17232,2020-01-02,morning,scattered clouds,12.0,5.0,NaN,NaN,NaN,NaN
4.0,WS_17233,2020-01-02,evening,scattered clouds,13.0,6.0,NaN,NaN,NaN,NaN


In [ ]:
mean_values = met_forecast_v2.groupby(by=['station_no', 'datetime']).mean().reset_index()
mean_values.sort_values(by=['station_no', 'datetime']).head()

,station_no,datetime,temp_max_day1,temp_min_day1,humidity_max_day1,humidity_min_day1,wind_dir_day1,wind_speed_day1
0,WS_17038,2020-01-02,10.0,7.0,NaN,NaN,NaN,NaN
1,WS_17038,2020-01-03,10.0,7.0,78.0,68.0,322.0,11.0
2,WS_17038,2020-01-04,10.0,7.0,85.0,65.0,41.0,11.0
3,WS_17038,2020-01-05,12.0,6.0,84.0,59.0,155.0,10.0
4,WS_17038,2020-01-06,13.0,9.0,60.0,50.0,161.0,16.0


*Categorical data*

In [ ]:
numerical_values = met_forecast_v2.describe().columns
non_num_df = met_forecast_v2[[x for x in met_forecast_v2.columns 
                              if x not in numerical_values]]
non_num_df.head()

,station_no,datetime,report_time,weather_day1
0.0,WS_17038,2020-01-02,evening,rain
1.0,WS_17038,2020-01-02,morning,rain
2.0,WS_17232,2020-01-02,evening,scattered clouds
3.0,WS_17232,2020-01-02,morning,scattered clouds
4.0,WS_17233,2020-01-02,evening,scattered clouds


In [ ]:
# 'Report_time' will be dropped
ohe_df = copy.deepcopy(non_num_df[['station_no', 'datetime']])
for x in range(1):
  temp = pd.get_dummies(non_num_df[f'weather_day{x+1}'], prefix=f'wd{x+1}')
  ohe_df = pd.concat([ohe_df, temp], axis=1)
ohe_df.head()

,station_no,datetime,wd1_few clouds,wd1_heavy rain,wd1_heavy rain showers,wd1_heavy snow,wd1_heavy thunderstorm with rain showers,wd1_light intensity shower rain,wd1_light rain showers,wd1_light snow,wd1_overcast clouds,wd1_rain,wd1_scattered clouds,wd1_sleet,wd1_snow,wd1_thunderstorm with heavy rain
0.0,WS_17038,2020-01-02,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1.0,WS_17038,2020-01-02,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2.0,WS_17232,2020-01-02,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3.0,WS_17232,2020-01-02,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4.0,WS_17233,2020-01-02,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [ ]:
# Dealing with two datetimes
ohe_df = ohe_df.groupby(by=['station_no', 'datetime'], as_index=False).agg(lambda x: 1 if sum(x) > 1 else sum(x))
ohe_df.head()

,station_no,datetime,wd1_few clouds,wd1_heavy rain,wd1_heavy rain showers,wd1_heavy snow,wd1_heavy thunderstorm with rain showers,wd1_light intensity shower rain,wd1_light rain showers,wd1_light snow,wd1_overcast clouds,wd1_rain,wd1_scattered clouds,wd1_sleet,wd1_snow,wd1_thunderstorm with heavy rain
0,WS_17038,2020-01-02,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,WS_17038,2020-01-03,0,0,0,0,0,0,0,0,1,1,0,0,0,0
2,WS_17038,2020-01-04,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,WS_17038,2020-01-05,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,WS_17038,2020-01-06,0,0,0,0,0,0,0,0,1,0,0,0,0,0


*Combining the numerical data with the categorical*

In [ ]:
modified_forecast_df = pd.merge(left=mean_values, right=ohe_df, on=['station_no', 'datetime'])
modified_forecast_df.head()

,station_no,datetime,temp_max_day1,temp_min_day1,humidity_max_day1,humidity_min_day1,wind_dir_day1,wind_speed_day1,wd1_few clouds,wd1_heavy rain,wd1_heavy rain showers,wd1_heavy snow,wd1_heavy thunderstorm with rain showers,wd1_light intensity shower rain,wd1_light rain showers,wd1_light snow,wd1_overcast clouds,wd1_rain,wd1_scattered clouds,wd1_sleet,wd1_snow,wd1_thunderstorm with heavy rain
0,WS_17038,2020-01-02,10.0,7.0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,WS_17038,2020-01-03,10.0,7.0,78.0,68.0,322.0,11.0,0,0,0,0,0,0,0,0,1,1,0,0,0,0
2,WS_17038,2020-01-04,10.0,7.0,85.0,65.0,41.0,11.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,WS_17038,2020-01-05,12.0,6.0,84.0,59.0,155.0,10.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,WS_17038,2020-01-06,13.0,9.0,60.0,50.0,161.0,16.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


#### On RL KPIS
* Using modified data provided by JSP
* Adding the forecast information from the nearest weather stations (from the previous day)
* Adding the KPI information from the previous day. (added by JSP)

In [ ]:
# To find nearest station
def find_nearest_stations(site_id: str, distances: pd.DataFrame, 
                          stations: list, k: int = 1) -> str:
  temp = distances[[site_id]].sort_values(by=[site_id])
  temp = temp.loc[[x for x in temp.index if x in stations]].head(k)
  return list(temp.index)

In [ ]:
%%time
# Around 1mins
xlsx = pd.ExcelFile('data-Test-2020-09-01.xlsx')
rl_kpis_mod = pd.read_excel(xlsx, sheet_name='rl-kpis', index_col=0, 
                       na_filter=True, convert_float=False)
rl_sites = pd.read_excel(xlsx, sheet_name='rl-sites', index_col=0, 
                       na_filter=True, convert_float=False)

CPU times: user 25.3 s, sys: 154 ms, total: 25.5 s
Wall time: 25.5 s


In [ ]:
# Some columns were dropped
rl_kpis_mod = rl_kpis_mod.drop(['direction', 'neid'], axis = 1) 
rl_kpis_mod.head()

,type,datetime,tip,mlid,mw_connection_no,site_id,polarization,card_type,adaptive_modulation,freq_band,link_length,severaly_error_second,error_second,unavail_second,avail_time,bbe,rxlevmax,scalibility_score,capacity,modulation,rlf
0.0,ENK,2020-01-02,NEAR,R0AP,314493.0,RL_9;SIP,Vertical,cardtype4,Enable,f3,2820.0,0.0,0.0,0.0,86400.0,0.0,-41.6,0.0,406.0,512QAM(QO),False
1.0,ENK,2020-01-02,FAR,R1AG,227760.0,RL_I7LB>,Vertical,cardtype4,Enable,f3,5898.0,0.0,0.0,0.0,86400.0,0.0,-35.4,0.0,203.0,512QAM,False
2.0,ENK,2020-01-02,NEAR,R1BA,345715.0,RL_I7LB>,Vertical,cardtype4,Enable,f3,5134.0,0.0,0.0,0.0,86400.0,0.0,-40.4,0.0,406.0,512QAM,False
3.0,ENK,2020-01-02,NEAR,R0TR,265780.0,RL_I7T?Q,Vertical,cardtype4,Enable,f5,611.0,0.0,0.0,0.0,86400.0,0.0,-39.8,0.0,203.0,512QAM,False
4.0,ENK,2020-01-02,NEAR,R3XD,335068.0,RL_I7Z?H,Vertical,cardtype1,Enable,f4,1873.0,0.0,0.0,0.0,86400.0,0.0,-25.0,0.0,406.0,512QAM,False


In [ ]:
# Extend dates a day ahead.
rl_kpis_mod_addendum = rl_kpis_mod.loc[rl_kpis_mod['datetime'] == '2020-01-15',:].copy()
rl_kpis_mod_addendum['datetime'] = [x + pd.Timedelta(days=1) for x in rl_kpis_mod_addendum['datetime']]
rl_kpis_mod = pd.concat([rl_kpis_mod,rl_kpis_mod_addendum])


In [ ]:
## Debug Cell
rl_kpis_mod

,type,datetime,tip,mlid,mw_connection_no,site_id,polarization,card_type,adaptive_modulation,freq_band,link_length,severaly_error_second,error_second,unavail_second,avail_time,bbe,rxlevmax,scalibility_score,capacity,modulation,rlf
0.0,ENK,2020-01-02,NEAR,R0AP,314493.0,RL_9;SIP,Vertical,cardtype4,Enable,f3,2820.0,0.0,0.0,0.0,86400.0,0.0,-41.6,0.0,406.0,512QAM(QO),False
1.0,ENK,2020-01-02,FAR,R1AG,227760.0,RL_I7LB>,Vertical,cardtype4,Enable,f3,5898.0,0.0,0.0,0.0,86400.0,0.0,-35.4,0.0,203.0,512QAM,False
2.0,ENK,2020-01-02,NEAR,R1BA,345715.0,RL_I7LB>,Vertical,cardtype4,Enable,f3,5134.0,0.0,0.0,0.0,86400.0,0.0,-40.4,0.0,406.0,512QAM,False
3.0,ENK,2020-01-02,NEAR,R0TR,265780.0,RL_I7T?Q,Vertical,cardtype4,Enable,f5,611.0,0.0,0.0,0.0,86400.0,0.0,-39.8,0.0,203.0,512QAM,False
4.0,ENK,2020-01-02,NEAR,R3XD,335068.0,RL_I7Z?H,Vertical,cardtype1,Enable,f4,1873.0,0.0,0.0,0.0,86400.0,0.0,-25.0,0.0,406.0,512QAM,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26676.0,ENK,2020-01-16,NEAR,R3NH,288419.0,RL_b;OGK,Vertical,cardtype4,Enable,f5,851.0,0.0,0.0,0.0,86400.0,0.0,-39.8,0.0,406.0,512QAM(QO),False
26677.0,ENK,2020-01-16,FAR,R3VZ,343850.0,RL_b;OGK,Vertical,cardtype4,Enable,f4,873.0,0.0,0.0,0.0,86400.0,0.0,-39.8,0.0,203.0,512QAM(QO),False
26678.0,ENK,2020-01-16,FAR,R3HF,336086.0,RL_b;SKB,Vertical,cardtype4,Enable,f5,464.0,0.0,0.0,0.0,86400.0,0.0,-39.5,0.0,200.0,512QAM,False
26679.0,ENK,2020-01-16,NEAR,T0KH,143854.0,RL_b?HR>,Vertical,cardtype4,Enable,f3,1326.0,0.0,0.0,0.0,86400.0,0.0,-39.8,0.0,406.0,512QAM,False


In [ ]:
%%time
# Around 2mins
# Forecast datetime should be - 1 day from the kpis datetime
rl_kpis_mod['forecast_datetime'] = [x - pd.Timedelta(days=1) for x in rl_kpis_mod['datetime']]

# Getting nearest station (just 1) based on the antennas - This might take a while
rl_kpis_mod['nearest_station'] = [find_nearest_stations(site_id, distances, stations)[0] for site_id in rl_kpis_mod['site_id']]

# Assuring the dates are timestamp type
modified_forecast_df['datetime'] = [pd.Timestamp(x) for x in modified_forecast_df['datetime']]
rl_kpis_mod['forecast_datetime'] = [pd.Timestamp(x) for x in rl_kpis_mod['forecast_datetime']]

# Modify name to merge
modified_forecast_df.rename(columns={'datetime':'forecast_datetime', 'station_no':'nearest_station'}, inplace=True) 

CPU times: user 2min 24s, sys: 38.6 ms, total: 2min 24s
Wall time: 2min 24s


In [ ]:
## KPI Historical (one day) (Added by JSP)
rl_kpis_history = rl_kpis_mod.copy()

# Dropping columns.
rl_kpis_history.drop(columns = ['forecast_datetime','nearest_station'],inplace=True)

# Assuring the dates are timestamp type
rl_kpis_history['datetime'] = [pd.Timestamp(x) for x in rl_kpis_history['datetime']]

# Adding with site data.
rl_kpis_history = rl_kpis_history.merge(rl_sites[['site_id','groundheight','clutter_class']], on='site_id')

# Renaming columns for merging.
rl_kpis_history.columns= ['history_{}'.format(column) for column in rl_kpis_history.columns]

rl_kpis_history

,history_type,history_datetime,history_tip,history_mlid,history_mw_connection_no,history_site_id,history_polarization,history_card_type,history_adaptive_modulation,history_freq_band,history_link_length,history_severaly_error_second,history_error_second,history_unavail_second,history_avail_time,history_bbe,history_rxlevmax,history_scalibility_score,history_capacity,history_modulation,history_rlf,history_groundheight,history_clutter_class
0,ENK,2020-01-02,NEAR,R0AP,314493.0,RL_9;SIP,Vertical,cardtype4,Enable,f3,2820.0,0.0,0.0,0.0,86400.0,0.0,-41.6,0.0,406.0,512QAM(QO),False,54.9587,OPEN LAND
1,ENK,2020-01-03,NEAR,R0AP,314493.0,RL_9;SIP,Vertical,cardtype4,Enable,f3,2820.0,0.0,0.0,0.0,86400.0,0.0,-41.6,0.0,406.0,512QAM(QO),False,54.9587,OPEN LAND
2,ENK,2020-01-06,NEAR,R0AP,314493.0,RL_9;SIP,Vertical,cardtype4,Enable,f3,2820.0,0.0,0.0,0.0,86400.0,0.0,-41.6,NaN,406.0,512QAM(QO),False,54.9587,OPEN LAND
3,ENK,2020-01-07,NEAR,R0AP,314493.0,RL_9;SIP,Vertical,cardtype4,Enable,f3,2820.0,0.0,0.0,0.0,86400.0,0.0,-41.6,NaN,406.0,512QAM(QO),False,54.9587,OPEN LAND
4,ENK,2020-01-13,NEAR,R0AP,314493.0,RL_9;SIP,Vertical,cardtype4,Enable,f3,2820.0,0.0,0.0,0.0,86400.0,0.0,-41.6,0.0,406.0,512QAM(QO),False,54.9587,OPEN LAND
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29445,ENK,2020-01-14,FAR,U3WB,1349170.0,RL_aENGH,Vertical,cardtype4,Enable,f3,3838.0,0.0,0.0,0.0,86400.0,0.0,-39.5,0.0,327.0,128QAM,False,501.0298,DENSE TREE
29446,NEC,2020-01-15,NEAR,U3BT,339402.0,RL_aENGH,Vertical,cardtype5,Enable,f4,4563.0,0.0,0.0,0.0,86400.0,0.0,-42.2,NaN,364.0,256QAM,False,501.0298,DENSE TREE
29447,ENK,2020-01-15,FAR,U3WB,1349170.0,RL_aENGH,Vertical,cardtype4,Enable,f3,3838.0,0.0,0.0,0.0,86400.0,0.0,-39.5,0.0,327.0,128QAM,False,501.0298,DENSE TREE
29448,NEC,2020-01-16,NEAR,U3BT,339402.0,RL_aENGH,Vertical,cardtype5,Enable,f4,4563.0,0.0,0.0,0.0,86400.0,0.0,-42.2,NaN,364.0,256QAM,False,501.0298,DENSE TREE


In [ ]:
## Debug Cell
rl_kpis_mod[rl_kpis_mod['rlf']== True]

,type,datetime,tip,mlid,mw_connection_no,site_id,polarization,card_type,adaptive_modulation,freq_band,link_length,severaly_error_second,error_second,unavail_second,avail_time,bbe,rxlevmax,scalibility_score,capacity,modulation,rlf,forecast_datetime,nearest_station
4570.0,ENK,2020-01-03,FAR,K7SW,316616.0,RL_SKUSO,Vertical,cardtype2,Disable,f5,308.0,0.0,1.0,201.0,86199.0,227.0,-29.4,NaN,154.0,128QAM,True,2020-01-02,WS_17232
6917.0,NEC,2020-01-04,FAR,K7IC,328406.0,RL_[7IR@,Vertical,cardtype5,Enable,f4,471.0,0.0,2.0,72603.0,86400.0,1385.0,-27.1,NaN,318.0,128QAM,True,2020-01-03,WS_18808
7240.0,ENK,2020-01-04,FAR,R2ZH,152829.0,RL_aKSNK,Vertical,cardtype4,Enable,f2,7601.0,55.0,179.0,364.0,86036.0,8432.0,-38.2,NaN,180.0,256QAM,True,2020-01-03,WS_17895
7472.0,ENK,2020-01-05,FAR,R2VA,310709.0,RL_IBLCV,Horizontal,cardtype1,Enable,f5,391.0,5.0,49.0,24434.0,61966.0,636.0,-39.8,NaN,406.0,512QAM,True,2020-01-04,WS_17310
10095.0,ENK,2020-01-06,NEAR,K3BZ,1381831.0,RL_IOI?I,Vertical,cardtype1,Enable,f3,27492.0,6.0,43.0,182.0,86218.0,2194.0,-47.3,0.00000,189.0,16QAM,True,2020-01-05,WS_18809
10566.0,ENK,2020-01-06,FAR,K3ZB,1381831.0,RL_MHCCV,Vertical,cardtype1,Enable,f3,27492.0,0.0,13.0,248.0,86152.0,723.0,-52.6,NaN,189.0,16QAM,True,2020-01-05,WS_18026
12733.0,ENK,2020-01-07,NEAR,R2WU,1378431.0,RL_KMHCA,Vertical,cardtype4,Enable,f1,3038.0,0.0,32.0,13958.0,72442.0,271.0,-42.0,NaN,74.0,128QAM,True,2020-01-06,WS_17310
15954.0,ENK,2020-01-09,NEAR,K7JC,223930.0,RL_IOQ?J,Vertical,cardtype4,Enable,f3,4442.0,2.0,20.0,191.0,86209.0,722.0,-30.4,0.00000,154.0,128QAM,True,2020-01-08,WS_18429
16392.0,NEC,2020-01-09,FAR,K7IC,328406.0,RL_[7IR@,Vertical,cardtype5,Enable,f4,471.0,0.0,1.0,36770.0,86400.0,1815.0,-26.3,NaN,318.0,128QAM,True,2020-01-08,WS_18808
16682.0,NEC,2020-01-10,FAR,K7SI,328281.0,RL_L?E?H,Vertical,cardtype5,Enable,f5,1298.0,0.0,1.0,52655.0,86400.0,2846.0,-34.7,NaN,364.0,256QAM,True,2020-01-09,WS_18808


In [ ]:
## Replacing Nearest Stations without Forecast

# weather station names.
ws = [i for i in distances.index if 'WS' in i] 

# distances to weather stations with forecast.
ws_with_forecast = np.intersect1d(rl_kpis_mod['nearest_station'].unique(),modified_forecast_df['nearest_station'].unique())
distances_ws_with_forecast = distances.loc[ws,ws_with_forecast].copy()

# weather stations without forecast.
ws_without_forecast = np.setdiff1d(rl_kpis_mod['nearest_station'].unique(),modified_forecast_df['nearest_station'].unique())
ws_replacement = [(i,distances_ws_with_forecast.loc[i].index[distances_ws_with_forecast.loc[i].argmin()]) for i in ws_without_forecast]

# replacing in dataset.
for i in ws_replacement:
  rl_kpis_mod.loc[rl_kpis_mod['nearest_station'] == i[0],'nearest_station'] = i[1]


*Merging the datasets*

In [ ]:
### Some values seems to be dropped
merged_df = pd.merge(rl_kpis_mod, modified_forecast_df, 
                      on=['nearest_station','forecast_datetime'],
                      validate='m:m')
merged_df

,type,datetime,tip,mlid,mw_connection_no,site_id,polarization,card_type,adaptive_modulation,freq_band,link_length,severaly_error_second,error_second,unavail_second,avail_time,bbe,rxlevmax,scalibility_score,capacity,modulation,rlf,forecast_datetime,nearest_station,temp_max_day1,temp_min_day1,humidity_max_day1,humidity_min_day1,wind_dir_day1,wind_speed_day1,wd1_few clouds,wd1_heavy rain,wd1_heavy rain showers,wd1_heavy snow,wd1_heavy thunderstorm with rain showers,wd1_light intensity shower rain,wd1_light rain showers,wd1_light snow,wd1_overcast clouds,wd1_rain,wd1_scattered clouds,wd1_sleet,wd1_snow,wd1_thunderstorm with heavy rain
0,ENK,2020-01-03,NEAR,R0AP,314493.0,RL_9;SIP,Vertical,cardtype4,Enable,f3,2820.0,0.0,0.0,0.0,86400.0,0.0,-41.6,0.00000,406.0,512QAM(QO),False,2020-01-02,WS_17300,14.0,8.0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,ENK,2020-01-03,NEAR,R3XD,335068.0,RL_I7Z?H,Vertical,cardtype1,Enable,f4,1873.0,0.0,0.0,0.0,86400.0,0.0,-26.0,0.00000,406.0,512QAM,False,2020-01-02,WS_17300,14.0,8.0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,ENK,2020-01-03,FAR,R0DK,306403.0,RL_I;YNL,Vertical,cardtype4,Enable,f4,1562.0,0.0,0.0,0.0,86400.0,0.0,-36.0,0.36419,406.0,512QAM(QO),False,2020-01-02,WS_17300,14.0,8.0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,ENK,2020-01-03,FAR,R0ED,313790.0,RL_I;YNL,Vertical,cardtype1,Enable,f3,3203.0,0.0,0.0,0.0,86400.0,0.0,-34.7,0.00000,406.0,512QAM,False,2020-01-02,WS_17300,14.0,8.0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,ENK,2020-01-03,FAR,R0GM,1377555.0,RL_I;YNL,Vertical,cardtype1,Enable,f3,4866.0,0.0,0.0,0.0,86400.0,0.0,-35.7,0.00000,456.0,1024QAM,False,2020-01-02,WS_17300,14.0,8.0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26819,NEC,2020-01-16,NEAR,U3EF,153015.0,RL_W<HPP,Vertical,cardtype5,Enable,f2,11729.0,0.0,0.0,0.0,86400.0,0.0,-41.8,NaN,247.0,2048QAM*,False,2020-01-15,WS_17775,10.0,1.0,91.0,71.0,248.0,7.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
26820,NEC,2020-01-16,NEAR,U3EJ,153015.0,RL_W<HPP,Horizontal,cardtype5,Enable,f2,11729.0,0.0,0.0,0.0,86400.0,0.0,-41.8,NaN,247.0,2048QAM*,False,2020-01-15,WS_17775,10.0,1.0,91.0,71.0,248.0,7.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
26821,NEC,2020-01-16,FAR,U3MF,1349689.0,RL_W<LGV,Vertical,cardtype5,Enable,f3,2141.0,0.0,0.0,0.0,86400.0,0.0,-38.7,NaN,77.0,128QAM,False,2020-01-15,WS_17775,10.0,1.0,91.0,71.0,248.0,7.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
26822,ENK,2020-01-16,NEAR,U3BG,290689.0,RL_W<ZCK,Vertical,cardtype4,Enable,f5,2048.0,0.0,0.0,0.0,86400.0,0.0,-39.5,0.00000,327.0,128QAM,False,2020-01-15,WS_17775,10.0,1.0,91.0,71.0,248.0,7.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [ ]:
## Some additional values seem to be dropped
merged_df = merged_df.merge(rl_kpis_history,left_on=['mlid','forecast_datetime'],right_on=['history_mlid','history_datetime'],validate='m:m')
merged_df

,type,datetime,tip,mlid,mw_connection_no,site_id,polarization,card_type,adaptive_modulation,freq_band,link_length,severaly_error_second,error_second,unavail_second,avail_time,bbe,rxlevmax,scalibility_score,capacity,modulation,rlf,forecast_datetime,nearest_station,temp_max_day1,temp_min_day1,humidity_max_day1,humidity_min_day1,wind_dir_day1,wind_speed_day1,wd1_few clouds,wd1_heavy rain,wd1_heavy rain showers,wd1_heavy snow,wd1_heavy thunderstorm with rain showers,wd1_light intensity shower rain,wd1_light rain showers,wd1_light snow,wd1_overcast clouds,wd1_rain,wd1_scattered clouds,wd1_sleet,wd1_snow,wd1_thunderstorm with heavy rain,history_type,history_datetime,history_tip,history_mlid,history_mw_connection_no,history_site_id,history_polarization,history_card_type,history_adaptive_modulation,history_freq_band,history_link_length,history_severaly_error_second,history_error_second,history_unavail_second,history_avail_time,history_bbe,history_rxlevmax,history_scalibility_score,history_capacity,history_modulation,history_rlf,history_groundheight,history_clutter_class
0,ENK,2020-01-03,NEAR,R0AP,314493.0,RL_9;SIP,Vertical,cardtype4,Enable,f3,2820.0,0.0,0.0,0.0,86400.0,0.0,-41.6,0.00000,406.0,512QAM(QO),False,2020-01-02,WS_17300,14.0,8.0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0,ENK,2020-01-02,NEAR,R0AP,314493.0,RL_9;SIP,Vertical,cardtype4,Enable,f3,2820.0,0.0,0.0,0.0,86400.0,0.0,-41.6,0.00000,406.0,512QAM(QO),False,54.9587,OPEN LAND
1,ENK,2020-01-03,NEAR,R3XD,335068.0,RL_I7Z?H,Vertical,cardtype1,Enable,f4,1873.0,0.0,0.0,0.0,86400.0,0.0,-26.0,0.00000,406.0,512QAM,False,2020-01-02,WS_17300,14.0,8.0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0,ENK,2020-01-02,NEAR,R3XD,335068.0,RL_I7Z?H,Vertical,cardtype1,Enable,f4,1873.0,0.0,0.0,0.0,86400.0,0.0,-25.0,0.00000,406.0,512QAM,False,91.0254,LOW-SPARSE URBAN
2,ENK,2020-01-03,FAR,R0DK,306403.0,RL_I;YNL,Vertical,cardtype4,Enable,f4,1562.0,0.0,0.0,0.0,86400.0,0.0,-36.0,0.36419,406.0,512QAM(QO),False,2020-01-02,WS_17300,14.0,8.0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0,ENK,2020-01-02,FAR,R0DK,306403.0,RL_I;YNL,Vertical,cardtype4,Enable,f4,1562.0,0.0,0.0,0.0,86400.0,0.0,-36.0,0.08242,406.0,512QAM(QO),False,56.9884,INDUSTRIAL & COMMERCIAL
3,ENK,2020-01-03,FAR,R0ED,313790.0,RL_I;YNL,Vertical,cardtype1,Enable,f3,3203.0,0.0,0.0,0.0,86400.0,0.0,-34.7,0.00000,406.0,512QAM,False,2020-01-02,WS_17300,14.0,8.0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0,ENK,2020-01-02,FAR,R0ED,313790.0,RL_I;YNL,Vertical,cardtype1,Enable,f3,3203.0,0.0,0.0,0.0,86400.0,0.0,-33.8,0.00000,406.0,512QAM,False,56.9884,INDUSTRIAL & COMMERCIAL
4,ENK,2020-01-03,FAR,R0GM,1377555.0,RL_I;YNL,Vertical,cardtype1,Enable,f3,4866.0,0.0,0.0,0.0,86400.0,0.0,-35.7,0.00000,456.0,1024QAM,False,2020-01-02,WS_17300,14.0,8.0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0,ENK,2020-01-02,FAR,R0GM,1377555.0,RL_I;YNL,Vertical,cardtype1,Enable,f3,4866.0,0.0,0.0,0.0,86400.0,0.0,-35.7,0.00000,456.0,1024QAM,False,56.9884,INDUSTRIAL & COMMERCIAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22553,NEC,2020-01-16,NEAR,U3EF,153015.0,RL_W<HPP,Vertical,cardtype5,Enable,f2,11729.0,0.0,0.0,0.0,86400.0,0.0,-41.8,NaN,247.0,2048QAM*,False,2020-01-15,WS_17775,10.0,1.0,91.0,71.0,248.0,7.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,NEC,2020-01-15,NEAR,U3EF,153015.0,RL_W<HPP,Vertical,cardtype5,Enable,f2,11729.0,0.0,0.0,0.0,86400.0,0.0,-41.8,NaN,247.0,2048QAM*,False,238.9611,BUILTUP-VILLAGE
22554,NEC,2020-01-16,NEAR,U3EJ,153015.0,RL_W<HPP,Horizontal,cardtype5,Enable,f2,11729.0,0.0,0.0,0.0,86400.0,0.0,-41.8,NaN,247.0,2048QAM*,False,2020-01-15,WS_17775,10.0,1.0,91.0,71.0,248.0,7.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,NEC,2020-01-15,NEAR,U3EJ,153015.0,RL_W<HPP,Horizontal,cardtype5,Enable,f2,11729.0,0.0,0.0,0.0,86400.0,0.0,-41.8,NaN,247.0,2048QAM*,False,238.9611,BUILTUP-VILLAGE
22555

In [ ]:
## Debug Cell
merged_df[merged_df['rlf']== True]

,type,datetime,tip,mlid,mw_connection_no,site_id,polarization,card_type,adaptive_modulation,freq_band,link_length,severaly_error_second,error_second,unavail_second,avail_time,bbe,rxlevmax,scalibility_score,capacity,modulation,rlf,forecast_datetime,nearest_station,temp_max_day1,temp_min_day1,humidity_max_day1,humidity_min_day1,wind_dir_day1,wind_speed_day1,wd1_few clouds,wd1_heavy rain,wd1_heavy rain showers,wd1_heavy snow,wd1_heavy thunderstorm with rain showers,wd1_light intensity shower rain,wd1_light rain showers,wd1_light snow,wd1_overcast clouds,wd1_rain,wd1_scattered clouds,wd1_sleet,wd1_snow,wd1_thunderstorm with heavy rain,history_type,history_datetime,history_tip,history_mlid,history_mw_connection_no,history_site_id,history_polarization,history_card_type,history_adaptive_modulation,history_freq_band,history_link_length,history_severaly_error_second,history_error_second,history_unavail_second,history_avail_time,history_bbe,history_rxlevmax,history_scalibility_score,history_capacity,history_modulation,history_rlf,history_groundheight,history_clutter_class
2436,ENK,2020-01-03,FAR,K7SW,316616.0,RL_SKUSO,Vertical,cardtype2,Disable,f5,308.0,0.0,1.0,201.0,86199.0,227.0,-29.4,NaN,154.0,128QAM,True,2020-01-02,WS_17232,12.0,5.0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,1,0,0,0,ENK,2020-01-02,FAR,K7SW,316616.0,RL_SKUSO,Vertical,cardtype2,Disable,f5,308.0,0.0,1.0,199.0,86201.0,1.0,-29.4,NaN,154.0,128QAM,False,3.700340e+05,HIGH-DENSE URBAN
3380,NEC,2020-01-04,FAR,K7IC,328406.0,RL_[7IR@,Vertical,cardtype5,Enable,f4,471.0,0.0,2.0,72603.0,86400.0,1385.0,-27.1,NaN,318.0,128QAM,True,2020-01-03,WS_17233,14.0,7.0,70.0,50.0,346.0,20.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,NEC,2020-01-03,FAR,K7IC,328406.0,RL_[7IR@,Vertical,cardtype5,Enable,f4,471.0,0.0,0.0,78821.0,86400.0,0.0,-26.9,NaN,318.0,128QAM,False,7.026000e+03,LOW-DENSE URBAN
4743,ENK,2020-01-05,FAR,R2VA,310709.0,RL_IBLCV,Horizontal,cardtype1,Enable,f5,391.0,5.0,49.0,24434.0,61966.0,636.0,-39.8,NaN,406.0,512QAM,True,2020-01-04,WS_17310,17.0,9.0,79.0,57.0,117.0,16.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,ENK,2020-01-04,FAR,R2VA,310709.0,RL_IBLCV,Horizontal,cardtype1,Enable,f5,391.0,0.0,0.0,0.0,86400.0,0.0,-39.5,NaN,406.0,512QAM,False,9.992500e+00,OPEN IN URBAN
6121,ENK,2020-01-06,NEAR,K3BZ,1381831.0,RL_IOI?I,Vertical,cardtype1,Enable,f3,27492.0,6.0,43.0,182.0,86218.0,2194.0,-47.3,0.000,189.0,16QAM,True,2020-01-05,WS_18427,11.0,4.0,90.0,77.0,85.0,15.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,ENK,2020-01-05,NEAR,K3BZ,1381831.0,RL_IOI?I,Vertical,cardtype1,Enable,f3,27492.0,0.0,0.0,0.0,86400.0,0.0,-47.3,0.00000,189.0,16QAM,False,2.060171e+06,SPARSE TREE
8416,ENK,2020-01-07,NEAR,R2WU,1378431.0,RL_KMHCA,Vertical,cardtype4,Enable,f1,3038.0,0.0,32.0,13958.0,72442.0,271.0,-42.0,NaN,74.0,128QAM,True,2020-01-06,WS_17310,16.0,11.0,87.0,74.0,163.0,29.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,ENK,2020-01-06,NEAR,R2WU,1378431.0,RL_KMHCA,Vertical,cardtype4,Enable,f1,3038.0,0.0,0.0,0.0,86400.0,0.0,-42.3,NaN,74.0,128QAM,False,1.114007e+03,SPARSE TREE
11572,NEC,2020-01-09,FAR,K7IC,328406.0,RL_[7IR@,Vertical,cardtype5,Enable,f4,471.0,0.0,1.0,36770.0,86400.0,1815.0,-26.3,NaN,318.0,128QAM,True,2020-01-08,WS_17233,11.0,5.0,73.0,53.0,7.0,33.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,NEC,2020-01-08,FAR,K7IC,328406.0,RL_[7IR@,Vertical,cardtype5,Enable,f4,471.0,0.0,0.0,45717.0,86400.0,0.0,-26.5,NaN,318.0,128QAM,False,7.026000e+03,LOW-DENSE URBAN
11660,ENK,2020-01-09,NEAR,K7JC,223930.0,RL_IOQ?J,Vertical,cardtype4,Enable,f3,4442.0,2.0,20.0,191.0,86209.0,722.0,-30.4,0.000,154.0,128QAM,True,2020-01-08,WS_18429,11.0,3.0,76.0,48.0,55.0,12.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,ENK,2020-01-08,NEAR,K7JC,223930.0,RL_IOQ?J,Vertical,cardtype4,Enable,f3,4442.0,0.0,0.0,0.0,86400.0,0.0,-30.4,0.00081,154.0,128QAM,False,1.489506e+06,LOW-DENSE URBAN
12115,NEC,2020-01-10,FAR,K7SI,328281.0,RL_L?E?H,Vertical,cardtype5,Enable,f5,1298.0,0.0,1.0,52655.0,86400.0,2846.0,-34.7,NaN,364.0,256QAM,True,2020-01-09,WS_17233,13.0,5.0,68.0,45.0,18.0,26.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,NEC,2020-01-09,FAR,K7SI,328281.0,R

In [ ]:
merged_df.to_csv('merged_data.csv')

#### On Merged dataset
* Dropping some links according to Amin's outlier RF Links analysis based on triangulation.
* Dropping some NA values
* Normalizing bbe and unavail_second
* Upsampling and downsampling based on the month

In [ ]:
to_delete = ['K3_MV', 'K3_DS', 'K3_AE', 'K3_CS', 'K3_DL']
merged_df = merged_df.loc[[x not in to_delete for x in merged_df['mlid_agnostic']]]

KeyError: ignored

*Normalizing per link*

In [ ]:
mean_per_mlid = {mlid:merged_df[merged_df['mlid'] == mlid]['bbe'].mean() for mlid in list(set(merged_df['mlid']))}
std_per_mlid = {mlid:merged_df[merged_df['mlid'] == mlid]['bbe'].std() for mlid in list(set(merged_df['mlid']))}
merged_df['bbe_normalized'] = [(x - mean_per_mlid[mlid])/std_per_mlid[mlid] if std_per_mlid[mlid] > 0 else 0 for x, mlid in zip(merged_df['bbe'], merged_df['mlid'])]

In [ ]:
mean_per_mlid = {mlid:merged_df[merged_df['mlid'] == mlid]['unavail_second'].mean() for mlid in list(set(merged_df['mlid']))}
std_per_mlid = {mlid:merged_df[merged_df['mlid'] == mlid]['unavail_second'].std() for mlid in list(set(merged_df['mlid']))}
merged_df['unavail_second_normalized'] = [(x - mean_per_mlid[mlid])/std_per_mlid[mlid] if std_per_mlid[mlid] > 0 else 0 for x, mlid in zip(merged_df['unavail_second'], merged_df['mlid'])]

*Adding the month*

In [ ]:
merged_df['month'] = [pd.Timestamp(x).month for x in merged_df['datetime']]

*Handling mistaken entries* (Added by JSP)[To remove scalibility score = date].

In [ ]:
merged_df

,type,datetime,tip,mlid,mw_connection_no,site_id,polarization,card_type,adaptive_modulation,freq_band,link_length,severaly_error_second,error_second,unavail_second,avail_time,bbe,rxlevmax,scalibility_score,capacity,modulation,rlf,forecast_datetime,nearest_station,temp_max_day1,temp_min_day1,humidity_max_day1,humidity_min_day1,wind_dir_day1,wind_speed_day1,wd1_few clouds,wd1_heavy rain,wd1_heavy rain showers,wd1_heavy snow,wd1_heavy thunderstorm with rain showers,wd1_light intensity shower rain,wd1_light rain showers,wd1_light snow,wd1_overcast clouds,wd1_rain,wd1_scattered clouds,wd1_sleet,wd1_snow,wd1_thunderstorm with heavy rain,history_type,history_datetime,history_tip,history_mlid,history_mw_connection_no,history_site_id,history_polarization,history_card_type,history_adaptive_modulation,history_freq_band,history_link_length,history_severaly_error_second,history_error_second,history_unavail_second,history_avail_time,history_bbe,history_rxlevmax,history_scalibility_score,history_capacity,history_modulation,history_rlf,history_groundheight,history_clutter_class,month
0,ENK,2020-01-03,NEAR,R0AP,314493.0,RL_9;SIP,Vertical,cardtype4,Enable,f3,2820.0,0.0,0.0,0.0,86400.0,0.0,-41.6,0.00000,406.0,512QAM(QO),False,2020-01-02,WS_17300,14.0,8.0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0,ENK,2020-01-02,NEAR,R0AP,314493.0,RL_9;SIP,Vertical,cardtype4,Enable,f3,2820.0,0.0,0.0,0.0,86400.0,0.0,-41.6,0.00000,406.0,512QAM(QO),False,54.9587,OPEN LAND,1
1,ENK,2020-01-03,NEAR,R3XD,335068.0,RL_I7Z?H,Vertical,cardtype1,Enable,f4,1873.0,0.0,0.0,0.0,86400.0,0.0,-26.0,0.00000,406.0,512QAM,False,2020-01-02,WS_17300,14.0,8.0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0,ENK,2020-01-02,NEAR,R3XD,335068.0,RL_I7Z?H,Vertical,cardtype1,Enable,f4,1873.0,0.0,0.0,0.0,86400.0,0.0,-25.0,0.00000,406.0,512QAM,False,91.0254,LOW-SPARSE URBAN,1
2,ENK,2020-01-03,FAR,R0DK,306403.0,RL_I;YNL,Vertical,cardtype4,Enable,f4,1562.0,0.0,0.0,0.0,86400.0,0.0,-36.0,0.36419,406.0,512QAM(QO),False,2020-01-02,WS_17300,14.0,8.0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0,ENK,2020-01-02,FAR,R0DK,306403.0,RL_I;YNL,Vertical,cardtype4,Enable,f4,1562.0,0.0,0.0,0.0,86400.0,0.0,-36.0,0.08242,406.0,512QAM(QO),False,56.9884,INDUSTRIAL & COMMERCIAL,1
3,ENK,2020-01-03,FAR,R0ED,313790.0,RL_I;YNL,Vertical,cardtype1,Enable,f3,3203.0,0.0,0.0,0.0,86400.0,0.0,-34.7,0.00000,406.0,512QAM,False,2020-01-02,WS_17300,14.0,8.0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0,ENK,2020-01-02,FAR,R0ED,313790.0,RL_I;YNL,Vertical,cardtype1,Enable,f3,3203.0,0.0,0.0,0.0,86400.0,0.0,-33.8,0.00000,406.0,512QAM,False,56.9884,INDUSTRIAL & COMMERCIAL,1
4,ENK,2020-01-03,FAR,R0GM,1377555.0,RL_I;YNL,Vertical,cardtype1,Enable,f3,4866.0,0.0,0.0,0.0,86400.0,0.0,-35.7,0.00000,456.0,1024QAM,False,2020-01-02,WS_17300,14.0,8.0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0,ENK,2020-01-02,FAR,R0GM,1377555.0,RL_I;YNL,Vertical,cardtype1,Enable,f3,4866.0,0.0,0.0,0.0,86400.0,0.0,-35.7,0.00000,456.0,1024QAM,False,56.9884,INDUSTRIAL & COMMERCIAL,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22553,NEC,2020-01-16,NEAR,U3EF,153015.0,RL_W<HPP,Vertical,cardtype5,Enable,f2,11729.0,0.0,0.0,0.0,86400.0,0.0,-41.8,NaN,247.0,2048QAM*,False,2020-01-15,WS_17775,10.0,1.0,91.0,71.0,248.0,7.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,NEC,2020-01-15,NEAR,U3EF,153015.0,RL_W<HPP,Vertical,cardtype5,Enable,f2,11729.0,0.0,0.0,0.0,86400.0,0.0,-41.8,NaN,247.0,2048QAM*,False,238.9611,BUILTUP-VILLAGE,1
22554,NEC,2020-01-16,NEAR,U3EJ,153015.0,RL_W<HPP,Horizontal,cardtype5,Enable,f2,11729.0,0.0,0.0,0.0,86400.0,0.0,-41.8,NaN,247.0,2048QAM*,False,2020-01-15,WS_17775,10.0,1.0,91.0,71.0,248.0,7.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,NEC,2020-01-15,NEAR,U3EJ,153015.0,RL_W<HPP,Horizontal,cardtype5,Enable,f2,11729.0,0.0,0.0,0.0,86400.0,0.0,-41.8,NaN,247.0,2048QAM*,False,238.9611

In [ ]:
## Debug Cell


*NA values*

In [ ]:
# Check for NA values
merged_df.columns[merged_df.isnull().any()]

Index(['freq_band', 'scalibility_score', 'humidity_max_day1',
       'humidity_min_day1', 'wind_dir_day1', 'wind_speed_day1',
       'history_freq_band', 'history_scalibility_score',
       'history_clutter_class'],
      dtype='object')

In [ ]:
# 2442 entries are dropped #Changed to 460 when JSP was testing
merged_df['freq_band'] = merged_df['freq_band'].fillna('None')
merged_df['history_freq_band'] = merged_df['history_freq_band'].fillna('None')
merged_df['history_clutter_class'] = merged_df['history_clutter_class'].fillna('None')
merged_df['polarization'] = merged_df['polarization'].fillna('None')
merged_df['history_polarization'] = merged_df['history_polarization'].fillna('None')
merged_df['scalibility_score'] = merged_df['scalibility_score'].fillna(-1)
merged_df['history_scalibility_score'] = merged_df['history_scalibility_score'].fillna(-1)
df = merged_df.copy() #df = merged_df.dropna()
print(merged_df.shape[0] - df.shape[0])

0


In [ ]:
df.head()

,type,datetime,tip,mlid,mw_connection_no,site_id,polarization,card_type,adaptive_modulation,freq_band,link_length,severaly_error_second,error_second,unavail_second,avail_time,bbe,rxlevmax,scalibility_score,capacity,modulation,rlf,forecast_datetime,nearest_station,temp_max_day1,temp_min_day1,humidity_max_day1,humidity_min_day1,wind_dir_day1,wind_speed_day1,wd1_few clouds,wd1_heavy rain,wd1_heavy rain showers,wd1_heavy snow,wd1_heavy thunderstorm with rain showers,wd1_light intensity shower rain,wd1_light rain showers,wd1_light snow,wd1_overcast clouds,wd1_rain,wd1_scattered clouds,wd1_sleet,wd1_snow,wd1_thunderstorm with heavy rain,history_type,history_datetime,history_tip,history_mlid,history_mw_connection_no,history_site_id,history_polarization,history_card_type,history_adaptive_modulation,history_freq_band,history_link_length,history_severaly_error_second,history_error_second,history_unavail_second,history_avail_time,history_bbe,history_rxlevmax,history_scalibility_score,history_capacity,history_modulation,history_rlf,history_groundheight,history_clutter_class,month
0,ENK,2020-01-03,NEAR,R0AP,314493.0,RL_9;SIP,Vertical,cardtype4,Enable,f3,2820.0,0.0,0.0,0.0,86400.0,0.0,-41.6,0.00000,406.0,512QAM(QO),False,2020-01-02,WS_17300,14.0,8.0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0,ENK,2020-01-02,NEAR,R0AP,314493.0,RL_9;SIP,Vertical,cardtype4,Enable,f3,2820.0,0.0,0.0,0.0,86400.0,0.0,-41.6,0.00000,406.0,512QAM(QO),False,54.9587,OPEN LAND,1
1,ENK,2020-01-03,NEAR,R3XD,335068.0,RL_I7Z?H,Vertical,cardtype1,Enable,f4,1873.0,0.0,0.0,0.0,86400.0,0.0,-26.0,0.00000,406.0,512QAM,False,2020-01-02,WS_17300,14.0,8.0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0,ENK,2020-01-02,NEAR,R3XD,335068.0,RL_I7Z?H,Vertical,cardtype1,Enable,f4,1873.0,0.0,0.0,0.0,86400.0,0.0,-25.0,0.00000,406.0,512QAM,False,91.0254,LOW-SPARSE URBAN,1
2,ENK,2020-01-03,FAR,R0DK,306403.0,RL_I;YNL,Vertical,cardtype4,Enable,f4,1562.0,0.0,0.0,0.0,86400.0,0.0,-36.0,0.36419,406.0,512QAM(QO),False,2020-01-02,WS_17300,14.0,8.0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0,ENK,2020-01-02,FAR,R0DK,306403.0,RL_I;YNL,Vertical,cardtype4,Enable,f4,1562.0,0.0,0.0,0.0,86400.0,0.0,-36.0,0.08242,406.0,512QAM(QO),False,56.9884,INDUSTRIAL & COMMERCIAL,1
3,ENK,2020-01-03,FAR,R0ED,313790.0,RL_I;YNL,Vertical,cardtype1,Enable,f3,3203.0,0.0,0.0,0.0,86400.0,0.0,-34.7,0.00000,406.0,512QAM,False,2020-01-02,WS_17300,14.0,8.0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0,ENK,2020-01-02,FAR,R0ED,313790.0,RL_I;YNL,Vertical,cardtype1,Enable,f3,3203.0,0.0,0.0,0.0,86400.0,0.0,-33.8,0.00000,406.0,512QAM,False,56.9884,INDUSTRIAL & COMMERCIAL,1
4,ENK,2020-01-03,FAR,R0GM,1377555.0,RL_I;YNL,Vertical,cardtype1,Enable,f3,4866.0,0.0,0.0,0.0,86400.0,0.0,-35.7,0.00000,456.0,1024QAM,False,2020-01-02,WS_17300,14.0,8.0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0,ENK,2020-01-02,FAR,R0GM,1377555.0,RL_I;YNL,Vertical,cardtype1,Enable,f3,4866.0,0.0,0.0,0.0,86400.0,0.0,-35.7,0.00000,456.0,1024QAM,False,56.9884,INDUSTRIAL & COMMERCIAL,1


*Dealing with month's distribution*

In [ ]:
# Months distribution
print(pd.DataFrame.from_dict(Counter(df['month']), orient='index').sort_values(by=0))
month_dist = dict(Counter(df['month']))
print(month_dist)

       0
1  22558
{1: 22558}


In [ ]:
'''
* Upsample: December [12]
* Downsample: July [7], September [9], October [10]
'''
from statistics import mean 
middle_ground = mean([month_dist[k] for k in month_dist.keys() if k in [6,11,8]])
middle_ground = int(round(middle_ground, -1))
print(middle_ground)

7880


In [ ]:
# Downsampling July [7], September [9], October [10]
sampled_df = copy.deepcopy(df.loc[[x in [6, 8, 11] for x in df['month']]])
for m in [7, 9, 10]:
  temp = df.loc[df['month'] == m]
  temp = temp.sample(middle_ground)
  sampled_df = pd.concat([sampled_df, temp], axis = 0)

In [ ]:
# Upsampling December [12] - NAIVE APPROACH
december = df.loc[df['month'] == 12]
temp = december.sample(n=middle_ground, replace=True)
sampled_df = pd.concat([sampled_df, temp], axis = 0)

In [ ]:
Counter(sampled_df['month'])

Counter({6: 7414, 7: 7880, 8: 8000, 9: 7880, 10: 7880, 11: 8225, 12: 7880})

*Dealing with more categorical data*

In [ ]:
# One hot encoding
columns_to_ohe = ['type', 'tip', 'polarization', 'card_type', 'adaptive_modulation', 'freq_band', 'modulation',
                  'history_type', 'history_tip', 'history_polarization', 'history_card_type', 'history_adaptive_modulation', 'history_freq_band', 'history_modulation',
                  'history_clutter_class']
# df = copy.deepcopy(sampled_df) # Edited by JSP
df = copy.deepcopy(df)
for c in columns_to_ohe:
  temp = pd.get_dummies(df[c], prefix=c).astype('int')
  df = df.drop(columns=c)
  df = pd.concat([df, temp], axis=1)

In [ ]:
# df.to_csv('data_v2.csv', index=False) # Edited by JSP
df.to_csv('PREDICTION_NoSampling.csv', index=False)

## Predictions

In [ ]:
df = pd.read_csv('PREDICTION_NoSampling.csv')
df.head()
df

,datetime,mlid,mw_connection_no,site_id,link_length,severaly_error_second,error_second,unavail_second,avail_time,bbe,rxlevmax,scalibility_score,capacity,rlf,forecast_datetime,nearest_station,temp_max_day1,temp_min_day1,humidity_max_day1,humidity_min_day1,wind_dir_day1,wind_speed_day1,wd1_few clouds,wd1_heavy rain,wd1_heavy rain showers,wd1_heavy snow,wd1_heavy thunderstorm with rain showers,wd1_light intensity shower rain,wd1_light rain showers,wd1_light snow,wd1_overcast clouds,wd1_rain,wd1_scattered clouds,wd1_sleet,wd1_snow,wd1_thunderstorm with heavy rain,history_datetime,history_mlid,history_mw_connection_no,history_site_id,...,history_modulation_1024 QAM,history_modulation_1024QAM,history_modulation_128QAM,history_modulation_128QAM(Q),history_modulation_128QAM-XPIC,history_modulation_16QAM,history_modulation_16QAM(Q),history_modulation_2048QAM,history_modulation_2048QAM*,history_modulation_256QAM,history_modulation_256QAM(Q),history_modulation_256QAM(QO),history_modulation_32QAM,history_modulation_4QAM,history_modulation_512QAM,history_modulation_512QAM(QO),history_modulation_512QAML,history_modulation_512QAM_std*,history_modulation_64QAM,history_modulation_C-QPSK,history_clutter_class_AVERAGE-DENSE URBAN,history_clutter_class_AVERAGE-MEDIUM URBAN,history_clutter_class_AVERAGE-SPARSE URBAN,history_clutter_class_BUILTUP-VILLAGE,history_clutter_class_CITY WALLS,history_clutter_class_DENSE TREE,history_clutter_class_GREEN HOUSE,history_clutter_class_HIGH-DENSE URBAN,history_clutter_class_HIGH-ISOLATED-BUILDINGS,history_clutter_class_HIGH-MEDIUM URBAN,history_clutter_class_HIGH-SPARSE URBAN,history_clutter_class_INDUSTRIAL & COMMERCIAL,history_clutter_class_LOW-DENSE URBAN,history_clutter_class_LOW-MEDIUM URBAN,history_clutter_class_LOW-SPARSE URBAN,history_clutter_class_None,history_clutter_class_OPEN IN URBAN,history_clutter_class_OPEN LAND,history_clutter_class_SPARSE TREE,history_clutter_class_VERYHIGH-SPARSE BLOCK BUILDINGS
0,2020-01-03,R0AP,314493.0,RL_9;SIP,2820.0,0.0,0.0,0.0,86400.0,0.0,-41.6,0.00000,406.0,False,2020-01-02,WS_17300,14.0,8.0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2020-01-02,R0AP,314493.0,RL_9;SIP,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,2020-01-03,R3XD,335068.0,RL_I7Z?H,1873.0,0.0,0.0,0.0,86400.0,0.0,-26.0,0.00000,406.0,False,2020-01-02,WS_17300,14.0,8.0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2020-01-02,R3XD,335068.0,RL_I7Z?H,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,2020-01-03,R0DK,306403.0,RL_I;YNL,1562.0,0.0,0.0,0.0,86400.0,0.0,-36.0,0.36419,406.0,False,2020-01-02,WS_17300,14.0,8.0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2020-01-02,R0DK,306403.0,RL_I;YNL,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,2020-01-03,R0ED,313790.0,RL_I;YNL,3203.0,0.0,0.0,0.0,86400.0,0.0,-34.7,0.00000,406.0,False,2020-01-02,WS_17300,14.0,8.0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2020-01-02,R0ED,313790.0,RL_I;YNL,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,2020-01-03,R0GM,1377555.0,RL_I;YNL,4866.0,0.0,0.0,0.0,86400.0,0.0,-35.7,0.00000,456.0,False,2020-01-02,WS_17300,14.0,8.0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2020-01-02,R0GM,1377555.0,RL_I;YNL,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22553,2020-01-16,U3EF,153015.0,RL_W<HPP,11729.0,0.0,0.0,0.0,86400.0,0.0,-41.8,-1.00000,247.0,False,2020-01-15,WS_17775,10.0,1.0,91.0,71.0,248.0,7.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2020-01-15,U3EF,153015.0,RL_W<HPP,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0

In [ ]:
df['link_length'] = df['link_length'].fillna('-1')
df['humidity_max_day1'] = df['humidity_max_day1'].fillna('-1')
df['humidity_min_day1'] = df['humidity_min_day1'].fillna('-1')
df['wind_dir_day1'] = df['wind_dir_day1'].fillna('-1')
df['wind_speed_day1'] = df['wind_speed_day1'].fillna('-1')
df['history_link_length'] = df['history_link_length'].fillna('-1')

In [ ]:
pred = pd.read_csv('predicted_rlf.tsv',sep='\t',usecols=['mlid'])
pred_values = pred['mlid'].values
df = df.loc[(df['datetime'] == '2020-01-16') & (df['mlid'].isin(pred_values)),:]
df

,datetime,mlid,mw_connection_no,site_id,link_length,severaly_error_second,error_second,unavail_second,avail_time,bbe,rxlevmax,scalibility_score,capacity,rlf,forecast_datetime,nearest_station,temp_max_day1,temp_min_day1,humidity_max_day1,humidity_min_day1,wind_dir_day1,wind_speed_day1,wd1_few clouds,wd1_heavy rain,wd1_heavy rain showers,wd1_heavy snow,wd1_heavy thunderstorm with rain showers,wd1_light intensity shower rain,wd1_light rain showers,wd1_light snow,wd1_overcast clouds,wd1_rain,wd1_scattered clouds,wd1_sleet,wd1_snow,wd1_thunderstorm with heavy rain,history_datetime,history_mlid,history_mw_connection_no,history_site_id,...,history_modulation_1024 QAM,history_modulation_1024QAM,history_modulation_128QAM,history_modulation_128QAM(Q),history_modulation_128QAM-XPIC,history_modulation_16QAM,history_modulation_16QAM(Q),history_modulation_2048QAM,history_modulation_2048QAM*,history_modulation_256QAM,history_modulation_256QAM(Q),history_modulation_256QAM(QO),history_modulation_32QAM,history_modulation_4QAM,history_modulation_512QAM,history_modulation_512QAM(QO),history_modulation_512QAML,history_modulation_512QAM_std*,history_modulation_64QAM,history_modulation_C-QPSK,history_clutter_class_AVERAGE-DENSE URBAN,history_clutter_class_AVERAGE-MEDIUM URBAN,history_clutter_class_AVERAGE-SPARSE URBAN,history_clutter_class_BUILTUP-VILLAGE,history_clutter_class_CITY WALLS,history_clutter_class_DENSE TREE,history_clutter_class_GREEN HOUSE,history_clutter_class_HIGH-DENSE URBAN,history_clutter_class_HIGH-ISOLATED-BUILDINGS,history_clutter_class_HIGH-MEDIUM URBAN,history_clutter_class_HIGH-SPARSE URBAN,history_clutter_class_INDUSTRIAL & COMMERCIAL,history_clutter_class_LOW-DENSE URBAN,history_clutter_class_LOW-MEDIUM URBAN,history_clutter_class_LOW-SPARSE URBAN,history_clutter_class_None,history_clutter_class_OPEN IN URBAN,history_clutter_class_OPEN LAND,history_clutter_class_SPARSE TREE,history_clutter_class_VERYHIGH-SPARSE BLOCK BUILDINGS
19800,2020-01-16,R2LC,1375849.0,RL_I;YNL,400.0,0.0,0.0,0.0,86400.0,0.0,-28.2,0.0,456.0,False,2020-01-15,WS_17300,14.0,5.0,71,43,349,15,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2020-01-15,R2LC,1375849.0,RL_I;YNL,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
19802,2020-01-16,R2BG,1368965.0,RL_IADJJ,2389.0,0.0,0.0,0.0,86400.0,0.0,-39.5,0.0,160.0,False,2020-01-15,WS_17300,14.0,5.0,71,43,349,15,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2020-01-15,R2BG,1368965.0,RL_IADJJ,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
19872,2020-01-16,R1CR,292193.0,RL_LKEQI,2532.0,0.0,45.0,0.0,86400.0,84.0,-30.0,0.0,456.0,False,2020-01-15,WS_17300,14.0,5.0,71,43,349,15,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2020-01-15,R1CR,292193.0,RL_LKEQI,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
19904,2020-01-16,R0DF,356814.0,RL_T7L?J,1283.0,0.0,0.0,0.0,86400.0,0.0,-33.3,-1.0,247.0,False,2020-01-15,WS_17300,14.0,5.0,71,43,349,15,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2020-01-15,R0DF,356814.0,RL_T7L?J,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
19953,2020-01-16,R3LM,1371561.0,RL_IDI?P,769.0,0.0,0.0,0.0,86400.0,0.0,-22.5,0.0,200.0,False,2020-01-15,WS_18016,17.0,6.0,82,39,310,11,0,0,0,0,0,1,0,0,0,0,0,0,0,0,2020-01-15,R3LM,1371561.0,RL_IDI?P,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22405,2020-01-16,K7LZ,301640.0,RL_SKTN>,469.0,0.0,0.0,0.0,86400.0,0.0,-39.8,0.0,417.0,False,2020-01-15,WS_17232,15.0,7.0,78,52,61,13,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2020-01-15,K7LZ,301640.0,RL_SKTN>,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22419,2020-01-16

#### Decision Tree Classifier

*Import the Model*

In [ ]:
import joblib
dtree_clf = joblib.load('20201029_dtree_clf_v4_NoSampling')
pruned_tree_features = joblib.load('20201029_dtree_clf_v4_NoSampling_inputfeatures')

*Processing the data for the model*

In [ ]:
to_keep = pruned_tree_features.tolist()
to_keep.append('rlf')

test_df = df.loc[:,to_keep]

# Select the target
target = 'rlf'

In [ ]:
## Debug Cell
test_df

,rxlevmax,capacity,temp_max_day1,temp_min_day1,humidity_max_day1,humidity_min_day1,wind_dir_day1,wind_speed_day1,wd1_few clouds,wd1_heavy rain,wd1_heavy rain showers,wd1_heavy thunderstorm with rain showers,wd1_overcast clouds,wd1_scattered clouds,wd1_thunderstorm with heavy rain,history_link_length,history_unavail_second,history_bbe,history_rxlevmax,history_capacity,history_groundheight,polarization_Vertical,card_type_cardtype2,card_type_cardtype5,history_card_type_cardtype4,history_adaptive_modulation_Enable,history_freq_band_f5,history_modulation_128QAM,history_clutter_class_HIGH-ISOLATED-BUILDINGS,rlf
19800,-28.2,456.0,14.0,5.0,71,43,349,15,0,1,0,0,0,0,0,400.0,0.0,0.0,-28.2,456.0,5.698840e+01,1,0,0,0,1,0,0,0,False
19802,-39.5,160.0,14.0,5.0,71,43,349,15,0,1,0,0,0,0,0,2389.0,0.0,0.0,-39.5,160.0,7.197720e+01,1,0,0,0,1,0,1,0,False
19872,-30.0,456.0,14.0,5.0,71,43,349,15,0,1,0,0,0,0,0,2532.0,0.0,84.0,-30.0,456.0,9.204020e+01,1,0,0,0,1,0,0,0,False
19904,-33.3,247.0,14.0,5.0,71,43,349,15,0,1,0,0,0,0,0,1283.0,0.0,0.0,-33.3,247.0,2.966500e+00,1,0,1,0,1,1,0,0,False
19953,-22.5,200.0,17.0,6.0,82,39,310,11,0,0,0,0,0,0,0,769.0,0.0,0.0,-22.5,200.0,5.097730e+01,1,0,0,1,1,1,0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22405,-39.8,417.0,15.0,7.0,78,52,61,13,0,0,0,0,1,0,0,469.0,0.0,0.0,-39.8,417.0,7.900310e+05,1,0,0,1,1,0,0,0,False
22419,-37.6,74.0,15.0,7.0,78,52,61,13,0,0,0,0,1,0,0,13319.0,0.0,411.0,-37.6,74.0,3.120312e+06,1,0,0,1,1,0,1,0,False
22431,-29.4,154.0,15.0,7.0,78,52,61,13,0,0,0,0,1,0,0,308.0,0.0,0.0,-29.4,154.0,3.700340e+05,1,1,0,0,0,1,1,0,False
22493,-39.8,456.0,7.0,2.0,92,77,151,4,0,0,0,0,1,0,0,4632.0,0.0,0.0,-39.8,456.0,4.239783e+02,1,0,0,0,1,0,0,0,False


In [ ]:
x_pred = test_df.loc[:, test_df.columns != target].values
x_pred = np.array(x_pred).astype(float)


In [ ]:
# Predictions
list(zip(df['mlid'].values,dtree_clf.predict(x_pred)))
# test_df['mlid'].values
# ,

[('R2LC', 0.0),
 ('R2BG', 0.0),
 ('R1CR', 0.0),
 ('R0DF', 0.0),
 ('R3LM', 0.0),
 ('R0PY', 0.0),
 ('R0BX', 0.0),
 ('R1ZB', 0.0),
 ('R3CA', 0.0),
 ('R3SX', 0.0),
 ('R0YP', 0.0),
 ('R3GA', 0.0),
 ('R1DP', 0.0),
 ('R0MP', 0.0),
 ('R3LB', 0.0),
 ('R1VM', 0.0),
 ('K3KN', 0.0),
 ('K7GB', 0.0),
 ('K7FN', 0.0),
 ('K3DU', 0.0),
 ('K7BR', 0.0),
 ('K3CL', 0.0),
 ('K3AV', 0.0),
 ('K3BU', 0.0),
 ('R2OM', 0.0),
 ('R2AD', 0.0),
 ('R2HV', 0.0),
 ('R2PH', 0.0),
 ('R3MN', 0.0),
 ('R3OH', 0.0),
 ('R2VA', 0.0),
 ('R3NM', 0.0),
 ('R2XM', 0.0),
 ('R3KO', 0.0),
 ('R2JL', 0.0),
 ('R2JU', 0.0),
 ('R2XY', 0.0),
 ('R2EK', 0.0),
 ('R2MK', 0.0),
 ('R2AY', 0.0),
 ('R3FD', 0.0),
 ('R2JY', 0.0),
 ('K7LJ', 0.0),
 ('K7LT', 0.0),
 ('K7FV', 0.0),
 ('K3BI', 0.0),
 ('K7GN', 0.0),
 ('K7PK', 0.0),
 ('R6DZ', 0.0),
 ('R6OT', 0.0),
 ('R6LK', 0.0),
 ('R6BV', 0.0),
 ('U3QI', 0.0),
 ('U3CE', 0.0),
 ('U3FR', 0.0),
 ('R6KX', 0.0),
 ('R6VO', 1.0),
 ('R6VD', 0.0),
 ('R6JO', 0.0),
 ('R6TK', 0.0),
 ('U3RD', 0.0),
 ('T0NE', 0.0),
 ('U3VD'